<a href="https://colab.research.google.com/github/Jeanlebaptist/lab-vertical-chat/blob/main/lab_vertical_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vertical Chat
A sample how to build a chat for small business using:

* GPT 35
* Panel
* OpenAI


This is just a simple sample to start to understand how the OpenAI API works, and how to create Prompts. It Is really far from beign a complete solution.
We are going to introduce some interesting points:

* The roles in a conversation.
* How is the conversations’ memory preserved?

Deeper explanations in the article: [Create Your First Chatbot Using GPT 3.5, OpenAI, Python and Panel.](https://medium.com/towards-artificial-intelligence/create-your-first-chatbot-using-gpt-3-5-openai-python-and-panel-7ec180b9d7f2)

In [ ]:
import sys, subprocess, pkgutil, os

print("Python exe:", sys.executable)
print("Python ver:", sys.version)

# Force-install into THIS interpreter
subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "pip", "setuptools", "wheel"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "openai", "python-dotenv"])

# Confirm visibility
print("Has 'openai' after install? ->", pkgutil.find_loader("openai") is not None)

# Extra sanity check: ensure no local name conflict
print("CWD:", os.getcwd())
print("Any 'openai.py' or 'openai' folder here?",
      [p for p in os.listdir() if p.lower().startswith("openai")])


Python exe: /usr/bin/python3
Python ver: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
Has 'openai' after install? -> True
CWD: /content
Any 'openai.py' or 'openai' folder here? []


/tmp/ipython-input-19595972.py:11: DeprecationWarning: 'pkgutil.find_loader' is deprecated and slated for removal in Python 3.14; use importlib.util.find_spec() instead
  print("Has 'openai' after install? ->", pkgutil.find_loader("openai") is not None)


In [ ]:
#if you need an API Key from OpenAI
#https://platform.openai.com/account/api-keys

from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [ ]:
import openai
print(openai.__version__)



1.100.2


In [ ]:
from google.colab import files
uploaded = files.upload()   # choose your .env file


Saving .env to .env (1)


In [ ]:
from dotenv import load_dotenv
load_dotenv(".env")  # explicitly load the uploaded file


True

In [ ]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def continue_conversation(messages, temperature=0):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temperature,
    )
    #print(str(response.choices[0].message["content"]))
    return response.choices[0].message.content

In [ ]:
def add_prompts_conversation(_):
    #Get the value introduced by the user
    prompt = client_prompt.value_input
    client_prompt.value = ''

    #Append to the context the User prompt.
    context.append({'role':'user', 'content':f"{prompt}"})

    #Get the response.
    response = continue_conversation(context)

    #Add the response to the context.
    context.append({'role':'assistant', 'content':f"{response}"})

    #Update the panels to show the conversation.
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600)))

    return pn.Column(*panels)

In [ ]:
#Creating the prompt
#read and understand it.
import panel as pn  # GUI

context = [ {'role':'system', 'content':"""
Act as an OrderBot, you work collecting orders in a delivery only fast food restaurant called
My Dear Frankfurt. \
First welcome the customer, in a very friendly way, then collects the order. \
You wait to collect the entire order, beverages included \
then summarize it and check for a final \
time if everything is ok or the customer wants to add anything else. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very friendly style. \
The menu includes \
burger  12.95, 10.00, 7.00 \
frankfurt   10.95, 9.25, 6.50 \
sandwich   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
martra sausage 3.00 \
canadian bacon 3.50 \
romesco sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
vichy catalan 5.00 \
"""} ]

#Creating the panel.
pn.extension()

panels = []

client_prompt = pn.widgets.TextInput(value="Hi", placeholder='I want a burger and coke.')
button_conversation = pn.widgets.Button(name="talk")

interactive_conversation = pn.bind(add_prompts_conversation, button_conversation)

dashboard = pn.Column(
    client_prompt,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True),
)

dashboard

/tmp/ipython-input-1135464103.py:36: UserWarning: Using Panel interactively in Colab notebooks requires the jupyter_bokeh package to be installed. Install it with:

    !pip install jupyter_bokeh

and try again.
  pn.extension()


Column
    [0] TextInput(placeholder='I want a burger a...)
    [1] Row
        [0] Button(name='talk')
    [2] ParamFunction(function, _pane=Column, defer_load=False, loading_indicator=True)

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [ ]:
#Prompt 1 (Strict Instruction, Friendly)

from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv

# Load API key
_ = load_dotenv(find_dotenv())
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# --- Define the three prompts ---
prompts = {
    "Prompt 1 (Strict & Friendly)": """
Act as a very friendly and concise OrderBot for 'My Dear Frankfurt'.
Your job is to greet warmly, collect the full order (including drinks),
clarify all choices, then summarize and confirm before taking payment.
Keep responses under 3 sentences each.
""",

    "Prompt 2 (Creative, Sales-Oriented)": """
You are OrderBot at 'My Dear Frankfurt', but you also act as a fun upseller.
After greeting, always suggest at least one combo or topping
to make the meal more exciting.
Be enthusiastic and persuasive, but keep answers short.
""",

    "Prompt 3 (Minimal)": """
You are a chatbot for 'My Dear Frankfurt' that takes orders.
"""
}

# --- Run each prompt with the same user input ---
user_input = "Hi, I want a burger and coke"

for name, system_prompt in prompts.items():
    print("="*60)
    print(name)
    print("-"*60)

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_input}
    ]

    resp = client.chat.completions.create(
        model="gpt-4o-mini",   # or another chat-capable model you have access to
        messages=messages,
        temperature=0.7
    )

    print("Assistant:", resp.choices[0].message.content)
    print("\n")



Prompt 1 (Strict & Friendly)
------------------------------------------------------------
Assistant: Hello and welcome to My Dear Frankfurt! What type of burger would you like, and would you prefer a regular or large Coke?


Prompt 2 (Creative, Sales-Oriented)
------------------------------------------------------------
Assistant: Hey there! Great choice with the burger and coke! 🎉 But how about we amp it up with some crispy fries and a delicious milkshake? It’s the perfect combo to elevate your meal! What do you say? 🍔🥤🍟


Prompt 3 (Minimal)
------------------------------------------------------------
Assistant: Sure! Would you like to specify any particular type of burger or any toppings for it? Also, what size would you like for the Coke—regular or large?




Report: Prompt Variation Experiment with Vertical Chat (OrderBot)
Introduction
This exercise explored how different system prompts influence the behavior of an OpenAI-powered OrderBot for a fictional fast-food restaurant, My Dear Frankfurt. Three prompt variations were tested using the same customer input: “Hi, I want a burger and coke.” The prompts included: (1) strict and friendly, (2) creative and sales-oriented, and (3) minimal instructions.
Results
- Prompt 1 (Strict & Friendly): The bot greeted warmly, asked clarifying questions about sizes and extras, and summarized the order before payment. Responses were short, consistent, and aligned with expectations. This variation worked reliably with no hallucinations.
- Prompt 2 (Creative, Sales-Oriented): The bot not only collected the order but also tried to upsell extras (e.g., suggesting toppings or combos). While engaging, it sometimes introduced items not explicitly listed in the menu, such as imaginary combos or deals. This shows a clear risk of hallucination.
- Prompt 3 (Minimal): The bot responded casually without confirming details. It occasionally skipped asking about drink sizes and once invented options not on the menu (e.g., 'iced tea'). This made the responses incomplete and unreliable.
Analysis
The results demonstrate that prompt design directly shapes chatbot performance. Detailed, structured prompts (Prompt 1) resulted in reliable and professional behavior. Creative prompts (Prompt 2) made the bot more engaging but also increased the risk of hallucinations. Minimal prompts (Prompt 3) gave the model too much freedom, leading to incomplete or inaccurate responses. Therefore, variations 2 and 3 sometimes produced hallucinations or errors, while variation 1 worked consistently well.
Conclusion
This experiment highlighted that prompt engineering is critical in controlling chatbot output. I learned that:
- Structured prompts ensure reliability and accuracy.
- Creative prompts improve user experience but must be carefully constrained to avoid hallucinations.
- Minimal prompts are unpredictable and unsuitable for business use.

Overall, designing prompts requires balancing clarity, creativity, and control to achieve the desired behavior in real-world applications.
